In [4]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

PATH_TO_DATA = Path('drive/My Drive/sendy/data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!pip install polyline

In [0]:
import pandas as pd
import numpy as np
import json
import pickle
import polyline
from tqdm import tqdm_notebook
import seaborn as sns
sns.set()

In [0]:
with open(PATH_TO_DATA / 'preprocessed/coords.pkl', 'rb') as f:
    coords = pickle.load(f)

In [7]:
len(coords)

28269

In [0]:
train = pd.read_csv(PATH_TO_DATA / 'given/Train.csv')
test = pd.read_csv(PATH_TO_DATA / 'given/Test.csv')

In [13]:
train.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [22]:
len(polyline.decode(json.loads(coords[0])['routes'][0]['geometry']))

156

In [0]:
def getFeatures(coords):    
    coords_feat = dict()
    for i, coord in enumerate(coords):
        data = json.loads(coord)
        routes = data['routes'][0]
        # coords_length = len(polyline.decode(routes['geometry']))
        segments = routes['segments'][0]
        ascent = segments.get('ascent')
        avg_speed = segments.get('avgspeed')
        descent = segments.get('descent')
        detourfactor = segments.get('detourfactor')
        distance = segments.get('distance')
        duration = segments.get('duration')
        steps_length = len(segments['steps'])
        # coords_feat.setdefault('coords_length', [])
        coords_feat.setdefault('ascent', [])
        coords_feat.setdefault('avg_speed', [])
        coords_feat.setdefault('descent', [])
        coords_feat.setdefault('detourfactor', [])
        coords_feat.setdefault('distance', [])
        coords_feat.setdefault('steps_length', [])
        coords_feat.setdefault('duration', [])
        # coords_feat['coords_length'].append(coords_length)
        coords_feat['ascent'].append(ascent)
        coords_feat['avg_speed'].append(avg_speed)
        coords_feat['descent'].append(descent)
        coords_feat['detourfactor'].append(detourfactor)
        coords_feat['distance'].append(distance)
        coords_feat['steps_length'].append(steps_length)
        coords_feat['duration'].append(duration)
        # print(i)
    
    return coords_feat

In [0]:
coords_dct = getFeatures(coords)

In [0]:
df_coords = pd.DataFrame.from_dict(coords_dct)

In [58]:
df_coords.head()

,ascent,avg_speed,descent,detourfactor,distance,steps_length,duration
0,9.845833,21.45,11.495833,1.59,3.037,11,509.8
1,130.242525,18.67,53.240025,1.42,16.246,25,3133.3
2,31.738324,21.49,15.863324,1.71,3.274,12,548.4
3,111.500595,21.02,33.350595,1.49,7.259,39,1243.2
4,92.232147,21.34,77.855224,1.66,6.226,21,1050.3


In [0]:
with open(PATH_TO_DATA / 'preprocessed/train_coords.pkl', 'wb') as f:
    pickle.dump(df_coords.iloc[:train.shape[0], :], f)

with open(PATH_TO_DATA / 'preprocessed/test_coords.pkl', 'wb') as f:
    pickle.dump(df_coords.iloc[train.shape[0]:, :], f)